# Uma Heurística GRASP para o Problema da Mochila Quadrática 0-1
O problema da mochila quadrática 0-1 (QKP) consiste em maximizar uma função objetivo quadrática sujeita a uma restrição linear, onde as variáveis são binárias (0 ou 1). Ele possui aplicações em diversas áreas, como finanças (alocação de recursos e seleção de carteira de ações) e seleção de projetos independentes. Formalmente: 

Maximizar: 

    Z = XᵀQX = ∑∑ qᵢⱼxᵢxⱼ 

Sujeito a: 

    ∑ aᵢxᵢ ≤ b xᵢ ∈ {0, 1} para todo i

Onde: 

- Q = (qᵢⱼ) é uma matriz n×n de coeficientes de "lucro" (definida positiva) 
- a = (aᵢ) é um vetor de pesos/capacidades 
- b é a capacidade total da mochila 
- xᵢ indica se o item i está (1) ou não (0) na mochila

Este problema é classificado como NP-Hard, o que significa que encontrar soluções exatas para instâncias maiores é computacionalmente inviável. Diante dessa dificuldade, o artigo propõe uma heurística GRASP (Greedy Randomized Adaptive Search Procedure) para resolvê-lo em sua formulação QKP1, que não será abordada neste trabalho.


---

In [38]:
# ---------------------------- DEPENDENCIES ----------------------------
import random
import numpy as np

In [39]:
# ---------------------------- CONFIG ----------------------------
# Número de itens
n = 20 

# Matriz de lucros
Q = np.random.rand(n, n)

# Pesos dos itens
a = np.random.randint(1, 10, n)

# Capacidade máxima
b = 20

## Algoritmo Evolutivo Selecionado
Por se tratar de um problema com variáveis binárias, o **Algoritmo Genético Tradicional** com **Codificação Binária** consegue lidar bem com o problema, desde que tenha uma boa função de aptidão. Ademais, este algoritmo consegue explorar eficientemente o espaço de busca de soluções binárias cheio de ótimos locais, além de incorporar facilmente a restrição de capacidade.

Para as partes desse algoritmo, definimos:
- **Representação de um Indivíduo**: vetor binário de tamanho n;
- **Função de Aptidão**: XᵀQX, se aᵀX ≤ b;  XᵀQX − λ × max(0, Σ aᵢxᵢ − b), se aᵀX > b 
- **Cruzamento**: Torneio Binário
- **Recombinação**: Crossover Uniforme
- **Mutação**: Bit-Flip (verificando viabilidade)
- **Sobrevivência**: (μ + λ)


In [ ]:
# ---------------------------- AUX FUNCTIONS ----------------------------

# Gera um array aleatório de tamanho n com 0s e 1s (tanto faz)
def random_individual():
    return np.random.randint(0, 2, n)

# Verifica se o indíviduo gerado respeita o limite de capacidade da mochila
def is_viable(individual, weights, capacity):
    total_weight = np.sum(individual * weights)
    return total_weight <= capacity

# Gera um array aleatório de tamanho n com 0s e 1s (viavel)
def random_individual_viable(weights, capacity):
    new_individual = np.random.randint(0, 2, n)

    if is_viable(new_individual, weights, capacity):
        return new_individual
    
    while (is_viable(new_individual, weights, capacity) == False):
        new_individual = mutation(new_individual)

    return new_individual

# Calcula o valor de aptidão de um indivíduo
def check_fitness(individual, profit_matrix):
    individual = np.asarray(individual)
    profit_matrix = np.asarray(profit_matrix)

    assert individual.ndim == 1, "Individual deve ser vetor 1D"
    assert profit_matrix.ndim == 2, "Matriz de lucro deve ser 2D"
    assert profit_matrix.shape[0] == profit_matrix.shape[1], "Matriz deve ser quadrada"
    assert profit_matrix.shape[0] == individual.shape[0], "Tamanhos incompatíveis"
    
    n = len(individual)
    fitness = 0.0
    for i in range(n):
        for j in range(n):
            # XᵀQX
            fitness += individual[i] * profit_matrix[i][j] * individual[j]
    return fitness

# Calcula a aptidão do indivíduo, aplicando penalidade de peso
def calc_fitness(individual, profit_matrix, weights, capacity, penalty=1000):
    # calculating fitness without considering the weight limit
    fitness = check_fitness(individual, profit_matrix) # XᵀQX, if aTx ≤ b
    # now checking the capacity
    excess = np.sum(individual * weights) - capacity
    # if excessing then it's over for the betinha
    # penalty is really high so it will be unlikely to be selected to reproduce
    if excess > 0:
        fitness -= penalty * excess # XᵀQX - λ × max(0, ∑aᵢxᵢ - b), if aTx > b

    return fitness

# Faz a recombinação/cruzamento dos pais (Crossover Uniforme)
def crossover(parent1, parent2, num_children=2):
    # print("\ncrossover")
    # considering the (μ + λ) approach
    children = []
    for _ in range(num_children):
        child = []
    # basically 50/50 chance to choose one gene from one parent
    # if pick is 0, then will receive gene from parent1, parent2 otherwise
        for gene in range(len(parent1)):
            pick = random.uniform(0, 1)
            if pick < 0.5:
                child.append(parent1[gene])
            else:
                child.append(parent2[gene])

        children.append(child)

    return children

# Faz a mutação de um indivíduo (Bit-Flip)
def mutation(individual):
    # print("\nmutacao")
    # choosing a random position from the individual array
    i = np.random.randint(len(individual))
    # just flipping the bit (since it's 0 or 1 just doing it works just  fine)
    individual[i] = 1 - individual[i]
    return individual
    
# Faz a seleção para cruzamento (Torneio Binário)
def tournament_selection(population, profit_matrix, weights, capacity):
    # print("torneio\n")
    # selecting two random individuals
    individual1 = random.choice(population)
    individual2 = random.choice(population)

    # ensuring they're not the same dude
    # while np.array_equal(individual1, individual2):
    #    individual2 = random.choice(population)
    #    print(f"individuo 1: {individual1}\nindividuo 2:{individual2}")
        
    # get them fitnesses
    fitness1 = calc_fitness(individual1, profit_matrix, weights, capacity)
    fitness2 = calc_fitness(individual2, profit_matrix, weights, capacity)

    # checking who's the boss
    if (fitness1 > fitness2):
        return individual1
    
    elif (fitness1 < fitness2):
        return individual2
    
    else:
        return random.choice([individual1, individual2])

# Seleção para sobrevivência (μ + λ)
def surviving_selection(population_parents, population_children, N, profit_matrix, weights, capacity,):
    # print("\nsobrevivencia")
    # putting them together
    pop_total = population_parents + population_children
    
    # calc fitnesses for everyone
    fitnesses_parents = [calc_fitness(ind, profit_matrix, weights, capacity) for ind in population_parents]
    fitnesses_children = [calc_fitness(ind, profit_matrix, weights, capacity ) for ind in population_children]

    # putting fitnesses together again
    fit_total = fitnesses_parents + fitnesses_children

    # joining everyone
    pairs = list(zip(pop_total, fit_total))

    # ordering them according to their aptitude
    ord_pairs = sorted(pairs, key=lambda x: x[1], reverse=True)

    # getting the best N dudes
    best_dudes = ord_pairs[:N]

    # separating values
    new_pop = [ind for ind, fit in best_dudes]
    new_fitnesses = [fit for ind, fit in best_dudes]

    return new_pop, new_fitnesses
    

In [41]:
# ---------------------------- ALGORITHM ----------------------------
def genetic_algorithm(pop_size, num_generations, profit_matrix, weights, capacity, crossover_prob, mutation_prob):
    part = int(pop_size * 0.7)

    # Inicialização da população
    population_random = [random_individual() for _ in range(pop_size - part)]
    population_viable = [random_individual_viable(weights, capacity) for _ in range(part)]
    population = population_random + population_viable


    best_individual = max(population, key=lambda ind: calc_fitness(ind, profit_matrix, weights, capacity))
    best_fit = calc_fitness(best_individual, profit_matrix, weights, capacity)
    print(f"Inicialmente: Melhor solução: {best_individual}\nAptidao = {best_fit:.2f}")

    # Evolução
    for generation in range(num_generations):
        # print(f"Geração {generation}\n")
        fitnesses = [calc_fitness(ind, profit_matrix, weights, capacity) for ind in population]
        new_population = []

        while len(new_population) < pop_size:
            p1 = tournament_selection(population, profit_matrix, weights, capacity)
            p2 = tournament_selection(population, profit_matrix, weights, capacity)

            if random.random() < crossover_prob:
                children = crossover(p1, p2)
                for child in children:
                    if random.random() < mutation_prob:
                        child = mutation(child)
            else:
                children = [p1, p2]

            new_population.extend(children)

        population, fitnesses = surviving_selection(population, new_population, pop_size, profit_matrix, weights, capacity)

        # Atualização do melhor indivíduo
        gen_best = max(population, key=lambda ind: calc_fitness(ind, profit_matrix, weights, capacity))
        gen_best_fit = calc_fitness(gen_best, profit_matrix, weights, capacity)

        generation += 1

        if gen_best_fit > best_fit:
            best_individual = gen_best
            best_ind_readable = list(map(int, best_individual))
            best_fit = gen_best_fit
            print(f"Geração {generation}: Melhor solução: x = {best_ind_readable}\nf(x) = {best_fit}")

    return (best_individual, best_fit, generation)


### Execução
Para o algoritmo genético, é necessário também fazer a definição de alguns parâmetros de execução. Nesse caso, a **condição de parada** é dada pelo número máximo de **gerações** alcançado. Além disso, definimos também um número de **execuções** para fins comparativos. Outros parâmetros para definição seriam:
- **Tamanho da População**: define o número máximos de indivíduos (p) a persistirem em uma população (inicialmente e na sobrevivência);
- **Probabilidade de Crossover**: define as chances de ocorrer reprodução entre dois indivíduos em uma população (geralmente alta);
- **Probabilidade de Mutação**: define as chances de ocorrer mutação em um indivíduo gerado em uma população (geralmente baixa).

In [42]:
# ---------------------------- MAIN ----------------------------
p_size = 50
n_gen = 30
c_prob = 0.8
m_prob = 0.2

num_exec = 5
results = []

for i in range(num_exec):
    print(f"Excecucao {i}")
    best_individual, best_fit, generation = genetic_algorithm(pop_size=p_size, num_generations=n_gen, profit_matrix=Q, weights=a, capacity=b, crossover_prob=c_prob, mutation_prob=m_prob)
    results.append((generation, best_individual, best_fit))
    print("\n")

best_of_all =  max(results, key=lambda tup: tup[2])
print(f"🏆Geração {best_of_all[0]}\nMelhor de todas as execuções: f(x) = {best_of_all[2]}\nIndivíduo: {list(map(int, best_of_all[1]))}")

Excecucao 0
Inicialmente: Melhor solução: [0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 1 0 0 0 0]
Aptidao = 24.96
Geração 3: Melhor solução: x = [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
f(x) = 31.793116510410616


Excecucao 1
Inicialmente: Melhor solução: [0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 1 1 0 0 1]
Aptidao = 25.94
Geração 4: Melhor solução: x = [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1]
f(x) = 26.42445461091623
Geração 7: Melhor solução: x = [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1]
f(x) = 32.024079919638666


Excecucao 2
Inicialmente: Melhor solução: [0 0 0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 1 0 0]
Aptidao = 24.72
Geração 1: Melhor solução: x = [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0]
f(x) = 29.336580369275477
Geração 2: Melhor solução: x = [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0]
f(x) = 30.607318766790893
Geração 3: Melhor solução: x = [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0]
f(x) = 31.59488

---

## Referências Bibliográficas
Nogueira, R. T., Paula Jr., G. G. de, & Póvoa, C. L. R. (2003). UMA HEURÍSTICA GRASP PARA O PROBLEMA DA MOCHILA QUADRÁTICA 0-1. A Pesquisa Operacional e os Recursos Renováveis, XXXVSBPO, Natal-RN, 4 a 7 de novembro de 2003.